<a href="https://colab.research.google.com/github/Prabha11/pre-datastorm-v1-0/blob/master/pre_data_storm_NN_model_step1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd

#read train data set
df = pd.read_csv("drive/My Drive/dataset/credit_card_default_train.csv", index_col=0)
df.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1M,M,Graduate,Other,31-45,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
A3,1M,F,High School,Other,Less than 30,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0
A4,100K,F,High School,Single,31-45,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1
A5,200K,F,Graduate,Single,31-45,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
A6,1M,F,Graduate,Other,31-45,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [0]:
#read test data set
df_test = pd.read_csv("drive/My Drive/dataset/credit_card_default_test.csv", index_col=0)
df_test.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,,,,,,,,,,,,,,,,,,
A20170,500K,M,Other,Single,31-45,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991
A16887,200K,F,Other,Single,31-45,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620
A3430,400K,M,Other,Other,Less than 30,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060
A3696,1.5M,M,Graduate,Single,31-45,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0
A20474,100K,F,High School,Other,Less than 30,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#test data set pre processing

#convet to string balance limit to float
df_test.Balance_Limit_V1 = (df_test.Balance_Limit_V1.replace(r'[KMB]+$', '', regex=True).astype(float) * df_test.Balance_Limit_V1.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))
df_test.head()

#gender education marital age stringto int convert
df_test.Gender.replace(['M', 'F'], [1, 0], inplace=True)
df_test.EDUCATION_STATUS.replace(['Graduate', 'High School', 'Other'], [2, 1, 0], inplace=True)
df_test.MARITAL_STATUS.replace(['Other', 'Single'], [1, 0], inplace=True)
df_test.AGE.replace(['Less than 30', '31-45', '46-65','More than 65'], [3, 2, 1, 0], inplace=True)

df_test.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#train data set pre processing

#convet to string balance limit to float
df.Balance_Limit_V1 = (df.Balance_Limit_V1.replace(r'[KMB]+$', '', regex=True).astype(float) * df.Balance_Limit_V1.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))
df.head()

#gender education marital age stringto int convert
df.Gender.replace(['M', 'F'], [1, 0], inplace=True)
df.EDUCATION_STATUS.replace(['Graduate', 'High School', 'Other'], [2, 1, 0], inplace=True)

#df.PAY_JULY.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_AUG.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_SEP.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_OCT.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_NOV.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_DEC.replace([-2, -1], [11, 13], inplace=True)

df.MARITAL_STATUS.replace(['Other', 'Single'], [1, 0], inplace=True)
df.AGE.replace(['Less than 30', '31-45', '46-65','More than 65'], [3, 2, 1, 0], inplace=True)

df["PAY_TOT"] = (df.PAY_JULY	+ df.PAY_AUG + df.PAY_SEP	+ df.PAY_OCT + df.PAY_NOV	+ df.PAY_DEC)
df["DUE_TOT"] = (df.DUE_AMT_JULY + df.DUE_AMT_AUG + df.DUE_AMT_SEP + df.DUE_AMT_OCT + df.DUE_AMT_NOV + df.DUE_AMT_DEC)

df.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT,PAY_TOT,DUE_TOT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0,-6,200849
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0,-4,781048
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1,7,48240
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1,2,578900
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0,4,2613331


In [0]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [0]:
selected_col_names = ["PAY_JULY",	"PAY_AUG", "PAY_SEP", "PAY_OCT", "PAY_TOT", "NEXT_MONTH_DEFAULT"]

df_selected = df[selected_col_names]
train_set, test_set = split_train_test(df_selected, 0.2)

X_train = train_set.drop("NEXT_MONTH_DEFAULT",1)   #Feature Matrix for training set
y_train = train_set["NEXT_MONTH_DEFAULT"]          #Target Variable for training set

X_test = test_set.drop("NEXT_MONTH_DEFAULT",1)   #Feature Matrix for testing set
y_test = test_set["NEXT_MONTH_DEFAULT"]          #Target Variable for testing set

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24000 entries, A2 to A30000
Data columns (total 26 columns):
Balance_Limit_V1      24000 non-null float64
Gender                24000 non-null int64
EDUCATION_STATUS      24000 non-null int64
MARITAL_STATUS        24000 non-null int64
AGE                   24000 non-null int64
PAY_JULY              24000 non-null int64
PAY_AUG               24000 non-null int64
PAY_SEP               24000 non-null int64
PAY_OCT               24000 non-null int64
PAY_NOV               24000 non-null int64
PAY_DEC               24000 non-null int64
DUE_AMT_JULY          24000 non-null int64
DUE_AMT_AUG           24000 non-null int64
DUE_AMT_SEP           24000 non-null int64
DUE_AMT_OCT           24000 non-null int64
DUE_AMT_NOV           24000 non-null int64
DUE_AMT_DEC           24000 non-null int64
PAID_AMT_JULY         24000 non-null int64
PAID_AMT_AUG          24000 non-null int64
PAID_AMT_SEP          24000 non-null int64
PAID_AMT_OCT          24000 non-

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
"""
Implementing the neural network
5 layers
"""
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(5,)),
    keras.layers.Dense(10, activation=tf.nn.relu),
	  keras.layers.Dense(10, activation=tf.nn.sigmoid),
	  keras.layers.Dense(10, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=1)

test_loss, test_acc = model.evaluate(X_train, y_train)
print('Test accuracy:', test_acc)

Train on 19200 samples
Epoch 1/10
19200/19200 [==============================] - 43s 2ms/sample - loss: 0.4530 - acc: 0.8142
Epoch 2/10
19200/19200 [==============================] - 42s 2ms/sample - loss: 0.4462 - acc: 0.8184
Epoch 3/10
19200/19200 [==============================] - 42s 2ms/sample - loss: 0.4438 - acc: 0.8202
Epoch 4/10
19200/19200 [==============================] - 43s 2ms/sample - loss: 0.4432 - acc: 0.8206
Epoch 5/10
19200/19200 [==============================] - 42s 2ms/sample - loss: 0.4426 - acc: 0.8197
Epoch 6/10
19200/19200 [==============================] - 41s 2ms/sample - loss: 0.4420 - acc: 0.8207
Epoch 7/10
19200/19200 [==============================] - 41s 2ms/sample - loss: 0.4414 - acc: 0.8195
Epoch 8/10
19200/19200 [==============================] - 40s 2ms/sample - loss: 0.4416 - acc: 0.8206
Epoch 9/10
19200/19200 [==============================] - 43s 2ms/sample - loss: 0.4411 - acc: 0.8211
Epoch 10/10
19200/19200 [==============================] - 